# Books Metadata

In this notebook, we will acquire books data from their ISBNs using the APIs.

In [1]:
from pathlib import Path
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import time

In [2]:
metadata_root = Path("./metadata")
books_data_path = metadata_root / "books.csv"

## Data Clean Up

First step is to clean ISBNs data by adding leading zeros and drop empty and duplicate ISBNs. And the end of this section, we will have a clean list of ISBNs.

In [3]:
books_df = pd.read_csv(books_data_path)
books_df

,ISBN,book_id
0,0002005018,1
1,0374157065,3
2,0399135782,5
3,0440234743,18
4,0452264464,19
...,...,...
16594,786914041,248348
16595,62117378,247944
16596,1905294964,248214
16597,1937007588,247154


In [4]:
books_df = books_df.dropna(subset=["ISBN"])
books_df.loc[:, "ISBN"] = books_df["ISBN"].astype(str)
books_df

,ISBN,book_id
0,0002005018,1
1,0374157065,3
2,0399135782,5
3,0440234743,18
4,0452264464,19
...,...,...
16594,786914041,248348
16595,62117378,247944
16596,1905294964,248214
16597,1937007588,247154


In [5]:
books_df.loc[:, "ISBN"] = books_df["ISBN"].str.zfill(10)
books_df

,ISBN,book_id
0,0002005018,1
1,0374157065,3
2,0399135782,5
3,0440234743,18
4,0452264464,19
...,...,...
16594,0786914041,248348
16595,0062117378,247944
16596,1905294964,248214
16597,1937007588,247154


In [6]:
isbn_list = books_df["ISBN"].unique()
print(f"Number of unique ISBNs: {len(isbn_list)}")
isbn_list

Number of unique ISBNs: 15540


array(['0002005018', '0374157065', '0399135782', ..., '0062117378',
       '1905294964', '1937007588'], dtype=object)

## Acquire Books Metadata

In [7]:
BASE_URL = "https://openlibrary.org"

### Book Data

In [12]:
def fetch_books_data(isbn_list, batch_size=64, delay=5, max_retries=20):
    books_data = {}

    for i in tqdm(range(0, len(isbn_list), batch_size), desc="Fetching Books"):
        batch_isbns = isbn_list[i : i + batch_size]
        bibkeys = ",".join(["ISBN:" + isbn for isbn in batch_isbns])
        params = {
            "bibkeys": bibkeys,
            "format": "json",
            "jscmd": "details",
        }
        retry_count = 0
        while retry_count < max_retries:
            try:
                response = requests.get(f"{BASE_URL}/api/books", params=params)
                response.raise_for_status()
                data = response.json()
                books_data.update(data)
                break
            except requests.exceptions.RequestException as e:
                retry_count += 1
                print(f"error at index {i} (attempt {retry_count}): {e}")
                if retry_count == max_retries:
                    print(f"giving up on batch at index {i}.")
                else:
                    time.sleep(delay * retry_count)
        time.sleep(delay)

    return books_data

In [13]:
books_data = fetch_books_data(isbn_list)
with open(metadata_root / "books_data.json", "w", encoding="utf-8") as f:
    json.dump(books_data, f, ensure_ascii=False, indent=4)

Fetching Books:  39%|███▉      | 95/243 [20:10<1:14:14, 30.10s/it]

error at index 6080 (attempt 1): 504 Server Error: Gateway Time-out for url: https://openlibrary.org/api/books?bibkeys=ISBN%3A0449006581%2CISBN%3A038547167X%2CISBN%3A0060913983%2CISBN%3A0874411033%2CISBN%3A0345434471%2CISBN%3A1551669412%2CISBN%3A0451205448%2CISBN%3A0452284295%2CISBN%3A0425190749%2CISBN%3A0451197550%2CISBN%3A159184021X%2CISBN%3A0743442652%2CISBN%3A1575667223%2CISBN%3A1575667215%2CISBN%3A038081918X%2CISBN%3A0380816830%2CISBN%3A0425184315%2CISBN%3A0380814390%2CISBN%3A1551666189%2CISBN%3A0743219333%2CISBN%3A0671786601%2CISBN%3A0505524228%2CISBN%3A0380782340%2CISBN%3A0505524279%2CISBN%3A0312981848%2CISBN%3A0515133868%2CISBN%3A1551668270%2CISBN%3A0373226055%2CISBN%3A0821765221%2CISBN%3A0446403830%2CISBN%3A074342347X%2CISBN%3A0671729454%2CISBN%3A0671007718%2CISBN%3A0446364819%2CISBN%3A8478886451%2CISBN%3A8478885196%2CISBN%3A8478884955%2CISBN%3A8478884459%2CISBN%3A0449208648%2CISBN%3A044918336X%2CISBN%3A0393000567%2CISBN%3A0553211668%2CISBN%3A0394550846%2CISBN%3A0679737383%2CI

Fetching Books:  51%|█████     | 124/243 [34:15<56:14, 28.35s/it]  

error at index 7936 (attempt 1): 504 Server Error: Gateway Time-out for url: https://openlibrary.org/api/books?bibkeys=ISBN%3A0590254766%2CISBN%3A0590553232%2CISBN%3A0060953691%2CISBN%3A0553275569%2CISBN%3A0060195703%2CISBN%3A0553289586%2CISBN%3A0684809001%2CISBN%3A0380400634%2CISBN%3A0345424646%2CISBN%3A0451193237%2CISBN%3A0679403612%2CISBN%3A0505524090%2CISBN%3A0505523892%2CISBN%3A0743471393%2CISBN%3A1401301231%2CISBN%3A050552354X%2CISBN%3A0425190641%2CISBN%3A0505523752%2CISBN%3A0446611360%2CISBN%3A0451524764%2CISBN%3A0553379658%2CISBN%3A1580050751%2CISBN%3A0465014909%2CISBN%3A1551667940%2CISBN%3A0140295569%2CISBN%3A0452278120%2CISBN%3A0316107298%2CISBN%3A0517266555%2CISBN%3A0553801880%2CISBN%3A0399142185%2CISBN%3A0425137562%2CISBN%3A1581346085%2CISBN%3A0553575953%2CISBN%3A0399139818%2CISBN%3A1558747435%2CISBN%3A0380815575%2CISBN%3A0517595338%2CISBN%3A1578561256%2CISBN%3A0671681567%2CISBN%3A3596150981%2CISBN%3A0812515595%2CISBN%3A0515135739%2CISBN%3A0590484125%2CISBN%3A0743202694%2CI

Fetching Books:  64%|██████▍   | 156/243 [50:02<31:28, 21.71s/it]  

...

Fetching Books:  84%|████████▍ | 204/243 [1:38:25<36:43, 56.51s/it]

error at index 13056 (attempt 1): 504 Server Error: Gateway Time-out for url: https://openlibrary.org/api/books?bibkeys=ISBN%3A0452285453%2CISBN%3A9799822939%2CISBN%3A0486422453%2CISBN%3A1600102379%2CISBN%3A141654707X%2CISBN%3A0446615862%2CISBN%3A0739352350%2CISBN%3A0803738552%2CISBN%3A1595144358%2CISBN%3A0758216432%2CISBN%3A0385520697%2CISBN%3A0297854399%2CISBN%3A0425217299%2CISBN%3A0793551617%2CISBN%3A141651693X%2CISBN%3A0552152978%2CISBN%3A078684907X%2CISBN%3A1501132938%2CISBN%3A0393064646%2CISBN%3A0812993292%2CISBN%3A0198245971%2CISBN%3A0822210789%2CISBN%3A0670026328%2CISBN%3A0062224077%2CISBN%3A1400068312%2CISBN%3A0778329860%2CISBN%3A1402262272%2CISBN%3A0553213873%2CISBN%3A0446556815%2CISBN%3A1401341705%2CISBN%3A0062414216%2CISBN%3A0394298667%2CISBN%3A006218850X%2CISBN%3A067001821X%2CISBN%3A0451229738%2CISBN%3A0312377045%2CISBN%3A0312990960%2CISBN%3A057507681X%2CISBN%3A1476731535%2CISBN%3A0307268195%2CISBN%3A0307593312%2CISBN%3A044657922X%2CISBN%3A0486455599%2CISBN%3A1595542779%2C

Fetching Books:  88%|████████▊ | 214/243 [1:45:32<19:27, 40.26s/it]

error at index 13696 (attempt 1): 504 Server Error: Gateway Time-out for url: https://openlibrary.org/api/books?bibkeys=ISBN%3A0061935085%2CISBN%3A0586203192%2CISBN%3A0446694851%2CISBN%3A0449000621%2CISBN%3A0449010880%2CISBN%3A0380817934%2CISBN%3A0679776192%2CISBN%3A0743484274%2CISBN%3A1451693567%2CISBN%3A0743470125%2CISBN%3A1593080646%2CISBN%3A0140620192%2CISBN%3A0310210062%2CISBN%3A1400033721%2CISBN%3A0316726192%2CISBN%3A0552154288%2CISBN%3A0061992259%2CISBN%3A0380821214%2CISBN%3A034541800X%2CISBN%3A0142406252%2CISBN%3A1595144919%2CISBN%3A0778322858%2CISBN%3A0312949804%2CISBN%3A0802142443%2CISBN%3A0446696137%2CISBN%3A039474067X%2CISBN%3A0812536363%2CISBN%3A0449000753%2CISBN%3A0803731531%2CISBN%3A0062248162%2CISBN%3A0099268701%2CISBN%3A0385532458%2CISBN%3A9993911550%2CISBN%3A0140449183%2CISBN%3A0062012037%2CISBN%3A0198320043%2CISBN%3A1416599398%2CISBN%3A0545424925%2CISBN%3A0670026336%2CISBN%3A0099549344%2CISBN%3A0439925509%2CISBN%3A0375869026%2CISBN%3A142150331X%2CISBN%3A0312424442%2C

Fetching Books: 100%|██████████| 243/243 [1:58:10<00:00, 29.18s/it]


### Work Data

In [18]:
with open(metadata_root / "books_data.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)
print(len(books_data))

15488


In [19]:
def extract_work_keys(books_data):
    work_keys = set()
    for book_info in books_data.values():
        if "works" in book_info["details"] and book_info["details"]["works"]:
            for work in book_info["details"]["works"]:
                work_key = work.get("key")
                if work_key:
                    work_keys.add(work_key)
    return list(work_keys)

In [24]:
def fetch_works_data(work_keys, batch_size=32, delay=0.25):
    works_data = {}
    for i in tqdm(range(0, len(work_keys), batch_size), desc="Fetching Works"):
        batch_keys = work_keys[i : i + batch_size]
        urls = [f"{BASE_URL}{key}.json" for key in batch_keys]
        try:
            for url in urls:
                response = requests.get(url)
                response.raise_for_status()
                data = response.json()
                works_data[data["key"]] = data
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching works batch starting at index {i}: {e}")
    return works_data

In [25]:
work_keys = extract_work_keys(books_data)
print(len(work_keys))

12357


In [26]:
works_data = fetch_works_data(work_keys)
with open(metadata_root / "works_data.json", "w", encoding="utf-8") as f:
    json.dump(works_data, f, ensure_ascii=False, indent=4)

Fetching Works:   1%|          | 2/387 [01:08<3:40:33, 34.37s/it]


KeyboardInterrupt: 

### Author Data

In [ ]:
def extract_author_keys(books_data, works_data):
    author_keys = set()
    # Extract from books
    for book_info in books_data.values():
        if "authors" in book_info and book_info["authors"]:
            for author in book_info["authors"]:
                author_key = author.get("key")
                if author_key:
                    author_keys.add(author_key)
    # Extract from works
    for work_info in works_data.values():
        if "authors" in work_info and work_info["authors"]:
            for author in work_info["authors"]:
                author_key = author.get("author", {}).get("key")
                if author_key:
                    author_keys.add(author_key)
    return list(author_keys)

In [ ]:
def fetch_authors_data(author_keys, batch_size=32, delay=0.1):
    authors_data = {}
    for i in tqdm(range(0, len(author_keys), batch_size), desc="Fetching Authors"):
        batch_keys = author_keys[i : i + batch_size]
        urls = [f"{BASE_URL}{key}.json" for key in batch_keys]
        try:
            for url in urls:
                response = requests.get(url)
                response.raise_for_status()
                data = response.json()
                authors_data[data["key"]] = data
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching authors batch starting at index {i}: {e}")
    return authors_data

In [ ]:
author_keys = extract_author_keys(books_data, works_data)
authors_data = fetch_authors_data(author_keys, batch_size=192, delay=2)
with open(metadata_root / "authors_data.json", "w", encoding="utf-8") as f:
    json.dump(authors_data, f, ensure_ascii=False, indent=4)